In [34]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [35]:
### Load the trained model, scaler pickle,onehot
model=load_model('ANN_model.keras')

## load the encoder and scaler
with open('OHE_encoder.pkl','rb') as file:
    OHE_encoder = pickle.load(file)

with open('label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [36]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [37]:
# One-hot encode 'Geography'
geo_encoded = OHE_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=OHE_encoder.get_feature_names_out(['Geography']))
geo_encoded_df


/Applications/CODE/ETEANNDL/eteanndl_venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [38]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [39]:
# One-hot encode 'Geography'
geo_encoder=OHE_encoder.transform(input_df[['Geography']]).toarray()
geo_OHE_encoder_df = pd.DataFrame(geo_encoded, columns=OHE_encoder.get_feature_names_out(['Geography']))
geo_OHE_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [40]:
## Encode categorical variables
input_df['Gender']=label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [41]:
input_df = pd.concat([input_df,geo_OHE_encoder_df],axis=1)
input_df.drop(['Geography'],axis=1,inplace=True)
input_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [42]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [43]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step


array([[0.05176438]], dtype=float32)

In [47]:
prediction_proba = prediction[0][0]
if prediction_proba > 0.5:
    print(f'The customer is likely to churn and {prediction_proba} is prdiction')
else:
    print(f'The customer is likely to churn and {prediction_proba} is prdiction')

The customer is likely to churn and 0.05176437646150589 is prdiction
